In [81]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.impute import SimpleImputer

In [82]:
pd.set_option("display.max_columns", 50)

data_df = pd.read_csv("risk-train.txt", sep = '\t')
data_df

,ORDER_ID,CLASS,B_EMAIL,B_TELEFON,B_BIRTHDATE,FLAG_LRIDENTISCH,FLAG_NEWSLETTER,Z_METHODE,Z_CARD_ART,Z_CARD_VALID,Z_LAST_NAME,VALUE_ORDER,WEEKDAY_ORDER,TIME_ORDER,AMOUNT_ORDER,ANUMMER_01,ANUMMER_02,ANUMMER_03,ANUMMER_04,ANUMMER_05,ANUMMER_06,ANUMMER_07,ANUMMER_08,ANUMMER_09,ANUMMER_10,CHK_LADR,CHK_RADR,CHK_KTO,CHK_CARD,CHK_COOKIE,CHK_IP,FAIL_LPLZ,FAIL_LORT,FAIL_LPLZORTMATCH,FAIL_RPLZ,FAIL_RORT,FAIL_RPLZORTMATCH,SESSION_TIME,NEUKUNDE,AMOUNT_ORDER_PRE,VALUE_ORDER_PRE,DATE_LORDER,MAHN_AKT,MAHN_HOECHST
0,49917,no,yes,no,1/17/1973,yes,yes,check,?,5.2006,?,17.80,Saturday,9:13,1,406811,?,?,?,?,?,?,?,?,?,no,no,no,no,no,no,no,no,no,no,no,no,8,yes,0,0.00,?,?,?
1,49919,no,yes,yes,12/8/1970,no,no,credit_card,Visa,12.2007,yes,18.20,Wednesday,17:36,1,600953,?,?,?,?,?,?,?,?,?,no,no,no,no,no,no,no,no,no,yes,no,no,13,yes,0,0.00,?,?,?
2,49923,no,yes,no,4/3/1972,yes,no,check,?,12.2007,?,10.80,Friday,11:13,1,406310,?,?,?,?,?,?,?,?,?,no,no,no,no,no,no,no,no,no,no,no,no,3,yes,0,0.00,?,?,?
3,49924,no,no,yes,8/1/1966,yes,no,check,?,1.2007,?,54.50,Friday,2:07,1,307359,?,?,?,?,?,?,?,?,?,no,no,no,yes,no,no,no,no,no,no,no,no,11,no,4,75.72,5/12/2002,0,0
4,49927,no,yes,yes,12/21/1969,yes,no,credit_card,Eurocard,12.2006,yes,19.99,Tuesday,23:46,1,200767,?,?,?,?,?,?,?,?,?,no,no,no,no,no,no,no,no,no,no,no,no,16,yes,0,0.00,?,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,49821,no,yes,no,3/3/1981,yes,no,check,?,6.2005,?,18.20,Sunday,18:21,1,406859,?,?,?,?,?,?,?,?,?,no,no,no,no,no,no,no,no,no,no,no,no,10,no,1,40.83,3/9/2002,0,3
29996,49824,no,yes,no,2/21/1972,no,no,credit_card,Eurocard,5.2005,yes,12.60,Saturday,15:51,1,206507,?,?,?,?,?,?,?,?,?,no,no,no,no,no,no,no,no,no,no,no,no,10,no,2,52.84,6/3/2001,0,0
29997,49825,no,yes,no,6/11/1980,no,no,credit_card,Eurocard,11.2006,yes,32.80,Monday,10:37,1,500767,?,?,?,?,?,?,?,?,?,no,no,no,no,no,no,no,no,no,no,no,no,6,yes,0,0.00,?,?,?
29998,49828,no,yes,no,12/4/1980,no,no,credit_card,Visa,4.2005,yes,5.20,Sunday,1:52,1,502611,?,?,?,?,?,?,?,?,?,no,no,no,no,no,no,no,no,no,no,no,no,6,no,1,42.92,7/1/2001,0,0


### Column Description ###

1. ORDER_ID: Unique identifier for each order.
2. CLASS: Target attribute indicating payment risk; values are "yes" (high risk) and "no" (low risk).
3. B_EMAIL: Boolean indicating if an email is associated with the order.
4. B_TELEFON: Boolean indicating if a phone number is provided.
5. B_BIRTHDATE: Customer's birthdate.
6. FLAG_LRIDENTISCH: Flag indicating if specific identification checks are identical.
7. FLAG_NEWSLETTER: Boolean showing if the customer is subscribed to newsletters.
8. Z_METHODE: Payment method (e.g., check, credit card).
9. Z_CARD_ART: Type of card used, if applicable.
10. Z_CARD_VALID: Card validity date.
11. Z_LAST_NAME: Last name of the customer, may be anonymized.
12. VALUE_ORDER: Total value of the order.
13. WEEKDAY_ORDER: Day of the week the order was placed.
14. TIME_ORDER: Time when the order was placed.
15. AMOUNT_ORDER: Number of items in the order. 
16-25. ANUMMER_01 to ANUMMER_10: Product codes for items in the order. 
26-28. CHK_LADR, CHK_RADR, CHK_KTO: Checks related to the address, account, and other attributes. 
29-34. CHK_CARD, CHK_COOKIE, CHK_IP, FAIL_LPLZ, FAIL_LORT, FAIL_LPLZORTMATCH: Flags and validation checks, indicating potential mismatches or issues with data verification. 
35-36. FAIL_RPLZ, FAIL_RORT: Additional mismatch checks for residence details. 
37-41. SESSION_TIME, NEUKUNDE, AMOUNT_ORDER_PRE, VALUE_ORDER_PRE, DATE_LORDER: Information on session duration, if the customer is new, previous order amount and value, and last order date. 
42-43. MAHN_AKT, MAHN_HOECHST: Dunning actions and highest dunning level associated with the customer.

In [83]:
data_df.replace('?', pd.NA, inplace = True)

In [84]:
data_df.CLASS.unique()

array(['no', 'yes'], dtype=object)

In [85]:
data_df.head()

null_percentage = data_df.isnull().mean() * 100
null_percentage_table = null_percentage.reset_index()
null_percentage_table.columns = ['Column', 'Null_Percentage']
null_percentage_table

,Column,Null_Percentage
0,ORDER_ID,0.000000
1,CLASS,0.000000
2,B_EMAIL,0.000000
3,B_TELEFON,0.000000
4,B_BIRTHDATE,9.806667
5,FLAG_LRIDENTISCH,0.000000
6,FLAG_NEWSLETTER,0.000000
7,Z_METHODE,0.000000
8,Z_CARD_ART,62.180000
9,Z_CARD_VALID,0.000000


In [86]:
# Removing Null Columns

column_to_drop = null_percentage_table[null_percentage_table['Null_Percentage'] > 20]['Column'].tolist()
data_df.drop(columns = column_to_drop, inplace = True)

In [87]:
# Formatting Columns

data_df["Z_CARD_VALID"] = data_df["Z_CARD_VALID"].astype(str)
data_df[['MONTH', 'YEAR']] = data_df["Z_CARD_VALID"].astype(str).str.split('.', expand = True)

data_df["Z_CARD_VALID_NEW"] = data_df.apply(lambda x: pd.to_datetime(f'{x.YEAR}-{x.MONTH}-01') + pd.offsets.MonthEnd(0), axis = 1)

In [88]:
data_df["Z_CARD_VALID"] = data_df["Z_CARD_VALID_NEW"]
data_df.drop(columns = ['MONTH', 'YEAR', 'Z_CARD_VALID_NEW'], inplace = True)

In [89]:
data_df["TIME_ORDER"] = pd.to_datetime(data_df["TIME_ORDER"], format = '%H:%M', errors = 'coerce')
data_df["TIME_ORDER"] = data_df["TIME_ORDER"].dt.strftime('%H:%M')

data_df["B_BIRTHDATE"] = pd.to_datetime(data_df["B_BIRTHDATE"], errors = 'coerce')

data_df['VALUE_ORDER'] = data_df['VALUE_ORDER'].astype(float)

data_df['AMOUNT_ORDER'] = data_df['AMOUNT_ORDER'].astype(int)

data_df['SESSION_TIME'] = data_df['SESSION_TIME'].astype(int)

data_df['AMOUNT_ORDER_PRE'] = data_df['AMOUNT_ORDER_PRE'].astype(int)

data_df['VALUE_ORDER_PRE'] = data_df['VALUE_ORDER_PRE'].astype(float)

In [90]:
print(data_df.info())
data_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ORDER_ID           30000 non-null  int64         
 1   CLASS              30000 non-null  object        
 2   B_EMAIL            30000 non-null  object        
 3   B_TELEFON          30000 non-null  object        
 4   B_BIRTHDATE        27058 non-null  datetime64[ns]
 5   FLAG_LRIDENTISCH   30000 non-null  object        
 6   FLAG_NEWSLETTER    30000 non-null  object        
 7   Z_METHODE          30000 non-null  object        
 8   Z_CARD_VALID       30000 non-null  datetime64[ns]
 9   VALUE_ORDER        30000 non-null  float64       
 10  WEEKDAY_ORDER      30000 non-null  object        
 11  TIME_ORDER         29980 non-null  object        
 12  AMOUNT_ORDER       30000 non-null  int32         
 13  ANUMMER_01         30000 non-null  int64         
 14  CHK_LA

,ORDER_ID,CLASS,B_EMAIL,B_TELEFON,B_BIRTHDATE,FLAG_LRIDENTISCH,FLAG_NEWSLETTER,Z_METHODE,Z_CARD_VALID,VALUE_ORDER,WEEKDAY_ORDER,TIME_ORDER,AMOUNT_ORDER,ANUMMER_01,CHK_LADR,CHK_RADR,CHK_KTO,CHK_CARD,CHK_COOKIE,CHK_IP,FAIL_LPLZ,FAIL_LORT,FAIL_LPLZORTMATCH,FAIL_RPLZ,FAIL_RORT,FAIL_RPLZORTMATCH,SESSION_TIME,NEUKUNDE,AMOUNT_ORDER_PRE,VALUE_ORDER_PRE
0,49917,no,yes,no,1973-01-17,yes,yes,check,2006-05-31,17.80,Saturday,09:13,1,406811,no,no,no,no,no,no,no,no,no,no,no,no,8,yes,0,0.00
1,49919,no,yes,yes,1970-12-08,no,no,credit_card,2007-12-31,18.20,Wednesday,17:36,1,600953,no,no,no,no,no,no,no,no,no,yes,no,no,13,yes,0,0.00
2,49923,no,yes,no,1972-04-03,yes,no,check,2007-12-31,10.80,Friday,11:13,1,406310,no,no,no,no,no,no,no,no,no,no,no,no,3,yes,0,0.00
3,49924,no,no,yes,1966-08-01,yes,no,check,2007-01-31,54.50,Friday,02:07,1,307359,no,no,no,yes,no,no,no,no,no,no,no,no,11,no,4,75.72
4,49927,no,yes,yes,1969-12-21,yes,no,credit_card,2006-12-31,19.99,Tuesday,23:46,1,200767,no,no,no,no,no,no,no,no,no,no,no,no,16,yes,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,49821,no,yes,no,1981-03-03,yes,no,check,2005-06-30,18.20,Sunday,18:21,1,406859,no,no,no,no,no,no,no,no,no,no,no,no,10,no,1,40.83
29996,49824,no,yes,no,1972-02-21,no,no,credit_card,2005-05-31,12.60,Saturday,15:51,1,206507,no,no,no,no,no,no,no,no,no,no,no,no,10,no,2,52.84
29997,49825,no,yes,no,1980-06-11,no,no,credit_card,2006-11-30,32.80,Monday,10:37,1,500767,no,no,no,no,no,no,no,no,no,no,no,no,6,yes,0,0.00
29998,49828,no,yes,no,1980-12-04,no,no,credit_card,2005-04-30,5.20,Sunday,01:52,1,502611,no,no,no,no,no,no,no,no,no,no,no,no,6,no,1,42.92


In [91]:
for column in data_df.columns:
    unique_values = data_df[column].unique()
    print(f"Unique values in column '{column}': {unique_values}")

Unique values in column 'ORDER_ID': [49917 49919 49923 ... 49825 49828 49829]
Unique values in column 'CLASS': ['no' 'yes']
Unique values in column 'B_EMAIL': ['yes' 'no']
Unique values in column 'B_TELEFON': ['no' 'yes']
Unique values in column 'B_BIRTHDATE': <DatetimeArray>
['1973-01-17 00:00:00', '1970-12-08 00:00:00', '1972-04-03 00:00:00',
 '1966-08-01 00:00:00', '1969-12-21 00:00:00', '1975-01-20 00:00:00',
 '1978-02-03 00:00:00', '1985-07-21 00:00:00', '1953-05-21 00:00:00',
 '1978-10-02 00:00:00',
 ...
 '1978-10-15 00:00:00', '1973-06-06 00:00:00', '1940-08-01 00:00:00',
 '1970-03-28 00:00:00', '1965-12-15 00:00:00', '1950-01-04 00:00:00',
 '1981-02-27 00:00:00', '1934-06-28 00:00:00', '1981-03-03 00:00:00',
 '1958-03-15 00:00:00']
Length: 11143, dtype: datetime64[ns]
Unique values in column 'FLAG_LRIDENTISCH': ['yes' 'no']
Unique values in column 'FLAG_NEWSLETTER': ['yes' 'no']
Unique values in column 'Z_METHODE': ['check' 'credit_card' 'debit_note' 'debit_card']
Unique values

In [92]:
# Creating Columns and updating columns

data_df['AGE'] = data_df['B_BIRTHDATE'].apply(lambda x: (datetime.now() - x).days // 365)
data_df = data_df.drop(columns=['B_BIRTHDATE']) 
data_df.replace('no', 0, inplace = True)

In [93]:
data_df.replace('yes', 1, inplace = True)


C:\Users\kedar\AppData\Local\Temp\ipykernel_5024\3973022085.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_df.replace('yes', 1, inplace = True)


In [94]:
data_df = pd.get_dummies(data_df, columns=['Z_METHODE', 'WEEKDAY_ORDER'], drop_first=True)


In [95]:
data_df['TIME_ORDER'] = pd.to_datetime(data_df['TIME_ORDER'], format='%H:%M')
data_df['TIME_ORDER'] = data_df['TIME_ORDER'].apply(lambda x: x.hour * 60 + x.minute)

In [96]:
data_df['Z_CARD_VALID'] = pd.to_datetime(data_df['Z_CARD_VALID'], format='%Y-%m-%d')


data_df['DAYS_TO_CARD_VALID'] = (data_df['Z_CARD_VALID'] - datetime.now()).dt.days


data_df.drop(columns=['Z_CARD_VALID'], inplace=True)

In [97]:
data_df

,ORDER_ID,CLASS,B_EMAIL,B_TELEFON,FLAG_LRIDENTISCH,FLAG_NEWSLETTER,VALUE_ORDER,TIME_ORDER,AMOUNT_ORDER,ANUMMER_01,CHK_LADR,CHK_RADR,CHK_KTO,CHK_CARD,CHK_COOKIE,CHK_IP,FAIL_LPLZ,FAIL_LORT,FAIL_LPLZORTMATCH,FAIL_RPLZ,FAIL_RORT,FAIL_RPLZORTMATCH,SESSION_TIME,NEUKUNDE,AMOUNT_ORDER_PRE,VALUE_ORDER_PRE,AGE,Z_METHODE_credit_card,Z_METHODE_debit_card,Z_METHODE_debit_note,WEEKDAY_ORDER_Monday,WEEKDAY_ORDER_Saturday,WEEKDAY_ORDER_Sunday,WEEKDAY_ORDER_Thursday,WEEKDAY_ORDER_Tuesday,WEEKDAY_ORDER_Wednesday,DAYS_TO_CARD_VALID
0,49917,0,1,0,1,1,17.80,553.0,1,406811,0,0,0,0,0,0,0,0,0,0,0,0,8,1,0,0.00,51.0,False,False,False,False,True,False,False,False,False,-6732
1,49919,0,1,1,0,0,18.20,1056.0,1,600953,0,0,0,0,0,0,0,0,0,1,0,0,13,1,0,0.00,53.0,True,False,False,False,False,False,False,False,True,-6153
2,49923,0,1,0,1,0,10.80,673.0,1,406310,0,0,0,0,0,0,0,0,0,0,0,0,3,1,0,0.00,52.0,False,False,False,False,False,False,False,False,False,-6153
3,49924,0,0,1,1,0,54.50,127.0,1,307359,0,0,0,1,0,0,0,0,0,0,0,0,11,0,4,75.72,58.0,False,False,False,False,False,False,False,False,False,-6487
4,49927,0,1,1,1,0,19.99,1426.0,1,200767,0,0,0,0,0,0,0,0,0,0,0,0,16,1,0,0.00,54.0,True,False,False,False,False,False,False,True,False,-6518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,49821,0,1,0,1,0,18.20,1101.0,1,406859,0,0,0,0,0,0,0,0,0,0,0,0,10,0,1,40.83,43.0,False,False,False,False,False,True,False,False,False,-7067
29996,49824,0,1,0,0,0,12.60,951.0,1,206507,0,0,0,0,0,0,0,0,0,0,0,0,10,0,2,52.84,52.0,True,False,False,False,True,False,False,False,False,-7097
29997,49825,0,1,0,0,0,32.80,637.0,1,500767,0,0,0,0,0,0,0,0,0,0,0,0,6,1,0,0.00,44.0,True,False,False,True,False,False,False,False,False,-6549
29998,49828,0,1,0,0,0,5.20,112.0,1,502611,0,0,0,0,0,0,0,0,0,0,0,0,6,0,1,42.92,43.0,True,False,False,False,False,True,False,False,False,-7128


In [98]:
data_df.CLASS.unique()

array([0, 1], dtype=int64)

In [117]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression


In [118]:
model = LogisticRegression()

In [119]:
X = data_df.drop(columns=['CLASS','ORDER_ID'])
y = data_df['CLASS']
imputer = SimpleImputer(strategy='mean')  


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=6,stratify=y)
print(y.value_counts())
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

CLASS
0    28254
1     1746
Name: count, dtype: int64


In [120]:
model.fit(X_train,y_train)

e:\Anaconda\envs\py-ml-env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [121]:
y_pred = model.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_pred, labels=[0, 1])

In [122]:
print("Confusion Matrix:\n", conf_matrix)


Confusion Matrix:
 [[8476    0]
 [ 524    0]]


In [123]:
report = classification_report(y_test, y_pred, labels=[0, 1], zero_division=1)

print("\nClassification Report:\n", report)



Classification Report:
               precision    recall  f1-score   support

           0       0.94      1.00      0.97      8476
           1       1.00      0.00      0.00       524

    accuracy                           0.94      9000
   macro avg       0.97      0.50      0.49      9000
weighted avg       0.95      0.94      0.91      9000



In [124]:
import sklearn.metrics as met

In [127]:
met.accuracy_score(y_test,y_pred)

0.9417777777777778

In [ ]:
met.precision_score(y_test,y_pred,zero_division=True)

1.0